This program reads a tab separated file containing the author name affiliation, and generate the coordinates corresponding to their institude.

One line of the file looks like this:     
`Adam	Y.	Université de Liège	Liège	Belgium`      
The city column was added to make easier the localisation.

For each year, two output files are created:
1. a file storing only the coordinates, that can be used directly with leaflet (see [leaflet](./leaflet) folder) 
2. a file that is the same as the input file but with 2 extra columns for the coordinates.

If the coordinates cannot be found for some reasons, a fill value (e.g. -999) is used, and the coordinates will have to be found "manually", i.e., using other resources.

In [9]:
import os
import glob
import logging
from collections import Counter
from liegecolloquium import Participant
from importlib import reload

# Configuration

In [19]:
datadir = "../data/2process/"
outputdir = "../data/"
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [25]:
filelist = sorted(glob.glob(os.path.join(datadir, "ParticipantList-2014.tsv")))
logging.info("Working on {0} file(s)".format(len(filelist)))

INFO:root:Working on 1 file(s)


In [26]:
participantlist = []
locationlist = []

for datafiles in filelist:
    logging.info("Working on file {0}".format(datafiles))
    outputbasename = os.path.basename(datafiles).replace(".tsv", ".dat")
    coordbasename = os.path.basename(datafiles).replace(".tsv", ".coord")
    outputfile = os.path.join(outputdir, outputbasename)
    coordfile = os.path.join(outputdir, coordbasename)
    logging.info("Writing info in file {0}".format(outputfile))
    logging.info("Writing coordinates in file {0}".format(outputfile))
    
    with open(coordfile, 'w') as cf:
        cf.write("var coords = [\n")

    with open(datafiles, 'r') as f:
        for line in f:
            
            if line == '\n':
                logger.debug("Empty line")
            else:
                l = line.rstrip().split('\t')
                participant = Participant(l[0], l[1], l[2], l[3], l[4])
                logger.info("Working with {0}".format(participant))

                # Modify country name via dictionary
                participant.replace_country()

                # Find location ; check if in the list before using geopy
                loc1 = ", ".join((participant.affiliation, 
                                  participant.city, 
                                  participant.country))
                # logger.debug(loc1)

                if loc1 in locationlist:
                    logger.debug("Already in the list")
                    ind = locationlist.index(loc1)
                    logger.debug("Position in the list: {0}".format(ind))
                    participant.lat = participantlist[ind].lat
                    participant.lon = participantlist[ind].lon
                    participant.countryname = participantlist[ind].countryname
                    locationlist.append(locationlist.index(loc1))
                else:
                    logger.debug("Not in the list, use geolocator to get coordinates")
                    locationstring, location = participant.get_location()
                    locationlist.append(locationstring)

                # Write info to a text file
                participant.write_to(outputfile)

                # Write coordinates (only) into another file
                participant.write_coords_to(coordfile)

                # Append to list
                participantlist.append(participant)
            
    with open(coordfile, 'a') as cf:
        cf.write("]")
logger.info("\n Job finished")

INFO:root:Working on file ../data/2process/ParticipantList-2014.tsv
INFO:root:Writing info in file ../data/ParticipantList-2014.dat
INFO:root:Writing coordinates in file ../data/ParticipantList-2014.dat
INFO:root:Working with Participant Arnaud Abadie (Belgique)
INFO:root:Working with Participant Mohamed A.R. Abdel-Moati (Qatar)
INFO:root:Working with Participant Arturo Aguirre (France)
INFO:root:Working with Participant Ebrahim Mohd A S Al-Ansari (Qatar)
INFO:root:Working with Participant Adnane Al-Azri (Oman)
INFO:root:Working with Participant Elin Almroth-Rosell (Sweden)
INFO:root:Working with Participant Marie-Lou Bachelery (France)
INFO:root:Working with Participant Hermann Bange (Germany)
INFO:root:Working with Participant Jonathan Barnes (U.K.)
INFO:root:Working with Participant Jean-Marie Beckers (Belgium)
INFO:root:Working with Participant Nadezhda Berezina (Russia)
INFO:root:Working with Participant Nicolas Berne (Belgium)
INFO:root:Working with Participant Joan M. Bernhard (

INFO:root:Working with Participant Martin Schmid (Switzerland)
INFO:root:Working with Participant Andreas Schöl (Germany)
INFO:root:Working with Participant Robert Schwefel (Switzerland)
INFO:root:Working with Participant Dorina Seitaj (The Netherlands)
INFO:root:Working with Participant Ekaterina Serebrennikova (Russia)
INFO:root:Working with Participant Silke Severmann (U.S.A.)
INFO:root:Working with Participant Ravail Singh (Germany)
INFO:root:Working with Participant Caroline P. Slomp (The Netherlands)
INFO:root:Working with Participant Martina Sollai (The Netherlands)
INFO:root:Working with Participant Gaelle Speeckaert (Belgium)
INFO:root:Working with Participant Emil Stanev (Germany)
INFO:root:Working with Participant Lothar Stramma (Germany)
INFO:root:Working with Participant Fatimah Sulu-Gambari (The Netherlands)
INFO:root:Working with Participant Maciej Telszewski (Poland)
INFO:root:Working with Participant Anders Tengberg (Sweden)
INFO:root:Working with Participant Jeremy Te